List of import

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math 


Creation of a Gaussian Filter

The Gaussian filter is a type of filter that applies a Gaussian distribution to an image through convolution.

To create a Gaussian filter, we first apply the Gaussian formula including the required values of sigma and the size of our mask. This is implemented through the functions *gauss*, which returns the Gaussian function given sigma and x dimensions, and *createMask*, which returns the mask with the required values given the mask dimensions and sigma. At this point, the *gaussianFilter* function, which takes the image, mask size, and sigma as input, applies the filter through a 2D convolution.



In [ ]:
#Gaussian filter
def gauss(sigma,x):
    ''' 
    Input: sigma: Value of sigma for Gauss function, x: Centered value to apply Gauss

    Output: Gauss function applied to sigma and x
    '''
    gauss = 1/(math.sqrt(2*math.pi)*sigma)*math.e**(-(x**2/(2*sigma**2)))
    return gauss

def createMask(maskSize, sigma):
    ''' 
    Input: maskSize: Dimension for the mask, sigma: Value of sigma for Gauss function

    Output: Mask 
    '''
    mask = np.zeros([maskSize + 1, maskSize +1], dtype = float)
    centre = int(maskSize / 2) + 1
    for i in range(1, maskSize +1):
        for j in range(1, maskSize +1):
            dist = np.sqrt(np.power(abs(i-centre),2)+np.power(abs(j-centre),2))
            mask[i][j] = gauss(sigma, dist)
    return mask


def gaussianFilter(img = cv2.imread(noisyImgs[1],0), maskSize = 3, sigma = 15):
    mask = createMask(maskSize, sigma)
    conv=cv2.filter2D(img, ddepth=-1, kernel=mask)
    #plt.figure()
    #plt.imshow(conv,cmap="gray")
    return conv



Creation of a Median Filter

The median filter is a filter that, by averaging the pixels in the surrounding area and the pixel under consideration, reduces impulsive noise or other irregular noise patterns.

The median filter is simpler to implement than the Gaussian filter, as it takes portions of the image of the kernel size and, after vectorizing the portion, calculates the median and puts it in the reconstructed image at the i-j-th position.

In [ ]:
def medianFilter(img =cv2.imread(noisyImgs[5],0) , maskSize = 3):
    m, n = np.shape(img)
    median = np.zeros_like(img)
    for i in range(1,m-1):
      for j in range (1,n-1):
         l = img[i:i + maskSize, j:j + maskSize].ravel()
         median[i,j] = np.mean(l)
    #plt.figure()
    #plt.imshow(median,cmap="gray")
    return median

MRF Implementation for Denoising Filters

The proposed MRF implementation uses convolution to search for the penalty associated with each pixel. This solution allows for the calculation of all pixel penalties in a single operation, reducing the need for loops. This operation is performed for all color intensities in the channel (in the case study, only grayscale) and returns a three-dimensional array with dimensions of 256 x height x width. For each pixel combination of a height and width position, we need to find the minimum value. A future implementation involves a C-written function to improve performance, as they are currently too dependent on double for loops.

The energy matrix of the data and the penalty energy matrix are then added together according to the algorithm.

The *denoiseImage* function takes the image, a weight for the penalties, and an exponent for the matrix as input.

A working copy of the image, an intensity array, and a mask are then created. The mask is then multiplied by WEp (Weight Energy Penalty).

For each of the grayscale intensities, a plane of a three-dimensional matrix is created, proposing the current intensity. Once all the proposals are collected, the best energy of each pixel is chosen.

A future implementation can take into account load balancing across different processors as the search for minimum energy is independent.

In [ ]:
def denoiseImage(X, WEp=0.3, exp=0.9):
  ''' 
  input: X np array for an image, iter int for number of iteration, WEp weight for data penalty

  output: Denoised imaged

  '''
  Y = np.copy(X)
  intesities = np.arange(260)
  mask = np.array([[0,1,0],[1,0,1],[0,1,0]])
  mask = mask * WEp

  energyMatrix = []

  for i in intesities:
    matrix = Y-i
    conv = cv2.filter2D(matrix, ddepth=-1, kernel=mask)
    tryIntensityEp = conv
    tryIntensityEp = np.power(tryIntensityEp,exp)
    tryIntensityEd = matrix**2
    sum = tryIntensityEp + tryIntensityEd
    energyMatrix.append(sum)
  
  energyMatrix = np.asarray(energyMatrix)
   

  #questo è ok perché prendiamo l'indice che ha come significato l'intensità di colore
  x,y,z = np.shape(energyMatrix)
  for i in range(y):
    for j in range(z):
      Y[i,j] = np.argmin(energyMatrix[:,i,j])
      #Possibile ottimizzazione: scrivere una funzione che prende la matrice energia e la divida per il numero di processori e poi faccia multiprocessing
  
  energyMatrix=[]
  #Da ottimizzare questa parte di post processing, magari usando un modulo
  
  return Y.astype(dtype=np.uint8)
